In [1]:
%env LANGCHAIN_PROJECT=rag_book_02

env: LANGCHAIN_PROJECT=rag_book_02


In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain.vectorstores import Chroma
import os
from dotenv import load_dotenv

load_dotenv()

model = AzureChatOpenAI(
            azure_deployment=os.getenv("OPENAI_CHAT_DEPLOYMENT_NAME"),
            openai_api_type="azure",
            temperature=0.0,
        )

embeddings = AzureOpenAIEmbeddings(
            deployment=os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
            chunk_size=1,
            embedding_ctx_length=1000,)

In [3]:
# A simple example dataset
examples = [
    {
        "inputs": {
            "question": "What's the name of the singer?",
            "documents": [
                {
                    "metadata": {},
                    "page_content": "the long - awaited moment arrives. the moon is high in the sky, the stars are shining like diamonds, and the whole audience is excited! the band takes the stage... our friends play and have a blast. they are partying, jumping, and dancing! fred sings happily because all his friends are with him on this special day. the night is so magical in the forest of starry sky!\nhis great friend alfred von jurgen, known as vj, is among the very first to receive the invitation. vj is a hazel bear descended from the noble bear families of the north. he has explored many lands and learned ancient bear dialects. he is the most technologically advanced bear in his family because he studied at a. i. t., the famous animal institute of technology. in this school, he learned the secrets of the compu ter world and electronic music.\n\" original story conceived and written by elfi e lupi. the images have been modified based on images generated by artificial intelligence.."
                }
            ],
        },
        "outputs": {
            "label": "Fred is the singer.",
        },
    },
    {
        "inputs": {
            "question": "What's the name of the forrest?",
            "documents": [
                {
                    "metadata": {},
                    "page_content": "once upon a time, there was an enchanted forest called starry sky, inhabited by many cute and funny animals. along its paths, from all corners of the world, par rots, giraffes, sheep, bears, and many other furry and fea thered friends found shelter. the forest embraced them with its branches, offering them warm shelter and many new friends. its magic? giving animals happy days, full of surprises and ne w adventures."
                }
            ],
        },
        "outputs": {
            "label": "Starry Sky",
        },
    }
]

In [4]:
from langsmith import Client

client = Client()

dataset_name = f"ds-rag-eval"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    inputs=[e["inputs"] for e in examples], 
    outputs=[e["outputs"] for e in examples],
    dataset_id=dataset.id,
)

In [5]:
from langchain import chat_models, prompts
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.retriever import BaseRetriever
from langchain.schema.document import Document

class MyRetriever(BaseRetriever):
    def _get_relevant_documents(self, query, *, run_manager):
        return [Document(page_content="Example")]

# THIS IS EVALUATED 
response_synthesizer = (
    prompts.ChatPromptTemplate.from_messages(
        [
            ("system", "Respond using the following documents as context:\n{documents}"),
            ("user", "{question}")
        ]
    ) | model
)

chain = (
    {
        "documents": MyRetriever(),
        "question": RunnablePassthrough(),
    }
    | response_synthesizer
)

In [7]:
from langsmith.evaluation import RunEvaluator, EvaluationResult
from langchain.evaluation import load_evaluator

class FaithfulnessEvaluator(RunEvaluator):

    def __init__(self):
        self.evaluator = load_evaluator(
            "labeled_score_string", 
            criteria={"faithful": "How faithful is the submission to the reference context?"},
            normalize_by=10,
            llm=model,
        )

    def evaluate_run(self, run, example) -> EvaluationResult:
        res = self.evaluator.evaluate_strings(
            prediction=next(iter(run.outputs.values())),
            input=run.inputs["question"],
            # We are treating the documents as the reference context in this case.
            reference=example.inputs["documents"],
        )
        return EvaluationResult(key="labeled_criteria:faithful", **res)

In [8]:
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    eval_llm=model,
    evaluators=["qa"],
    custom_evaluators=[FaithfulnessEvaluator()],
    input_key="question",
)
results = client.run_on_dataset(
    llm_or_chain_factory=response_synthesizer,
    dataset_name=dataset_name,
    evaluation=eval_config,
)   

View the evaluation results for project 'aching-doctor-6' at:
https://smith.langchain.com/o/1a3b231d-e8b9-4837-8416-2403db894308/datasets/a2f55d1e-77d5-4fb7-b659-343a543315bf/compare?selectedSessions=76274d2c-511b-4461-998b-a983a4c786e8

View all tests for Dataset ds-rag-eval at:
https://smith.langchain.com/o/1a3b231d-e8b9-4837-8416-2403db894308/datasets/a2f55d1e-77d5-4fb7-b659-343a543315bf
[------------------------------------------------->] 2/2

## Running the chain 

In [10]:
CHROMA_PERSISTENT_DIR = "../../data/chroma"
CHROMA_COLLECTION_NAME = "book_eng"

vectorstore = Chroma(
            collection_name=CHROMA_COLLECTION_NAME,
            persist_directory=CHROMA_PERSISTENT_DIR,
            embedding_function=embeddings,
        )


In [11]:
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 3, "score_threshold": 0.9})
#retriever.get_relevant_documents("What's the name of the singer?")


In [12]:
from langchain import chat_models, prompts
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.retriever import BaseRetriever
from langchain.schema.document import Document

class MyRetriever(BaseRetriever):
    def _get_relevant_documents(self, query, *, run_manager):
        retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 3, "score_threshold": 0.9})        
        return retriever.get_relevant_documents(query)
        

# This is what we will evaluate
response_synthesizer = (
    prompts.ChatPromptTemplate.from_messages(
        [
            ("system", "Respond using the following documents as context:\n{documents}"),
            ("user", "{question}")
        ]
    ) | model
)

# Full chain below for illustration
chain = (
    {
        "documents": MyRetriever(),
        "question": RunnablePassthrough(),
    }
    | response_synthesizer
)

In [13]:
chain.invoke("What's the name of the singer?")

AIMessage(content='The name of the singer is Fred.')

## ChatPromptTemplate


add to rag eval
{
  "label": "Starry Sky"
}